In [5]:
import os

os.environ['KERAS_BACKEND'] = "torch"

# Dataset Generation
Our dataset is a corpus of reviews scrapped from the BGG API. <br /> 
In order to download the comments we make use of the ```bgg_corpus_service.py``` content.

BGG Does not directly provide a way to list all the games it has in archive therefore we used a dump created by the community (2024-08-18).

In [6]:
# /resources/2024-08-18.csv
corpus_file = "../data/corpus.preprocessed.csv"

In [7]:
import model.embeddings as embeddings

embeddings_model = embeddings.WordEmbedding(
    embeddings.LoadCorpusUtility(), max_vocab_size=10000, embedding_size=128,
    target_model_file="./../data/word-embeddings.model", corpus_file=corpus_file
)

# Custom Dataloaders Definitions
To train the model we require a way to get elements of our dataset. I designed 3 classes for this:

In [8]:
import dataset

lazy = dataset.LazyCommentDataset("../data/corpus.preprocessed.csv")
dt = dataset.CommentDataset(embeddings_model.get_vocab(), "../data/corpus.preprocessed.csv")

Pandas Apply:   0%|          | 0/1939904 [00:00<?, ?it/s]

477790308
<class 'pandas.core.series.Series'>
RangeIndex: 1939904 entries, 0 to 1939903
Series name: comments
Non-Null Count    Dtype 
--------------    ----- 
1939904 non-null  object
dtypes: object(1)
memory usage: 455.7 MB
None


Without preprocessing on dataset: 379211268 <br />
If the dataset is processed already: 677671688 (Twice the size) <br />

In [9]:
from torch.utils.data import DataLoader

dt_dataloader = DataLoader(dt, batch_size=32, shuffle=True, collate_fn=lambda x: x)
lazy_dataloader = DataLoader(lazy, batch_size=32, shuffle=True, collate_fn=lambda x: x)

In [10]:
dt_iterator = iter(dt_dataloader)
lazy_iterator = iter(lazy_dataloader)

# PositiveNegativeCommentGeneratorDataset

In [11]:
from dataset import PositiveNegativeCommentGeneratorDataset

embeddings_model.load_model()
embeddings_model.get_vocab()
vocabulary = embeddings_model.model.wv.key_to_index

ds = PositiveNegativeCommentGeneratorDataset("./../data/corpus.preprocessed.csv", vocabulary, 10)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: Data must be 1-dimensional, got ndarray of shape (1000, 1017) instead